In [3]:
import pandas as pd
import numpy as np

from google.cloud import bigquery

client = bigquery.Client()

In [7]:
start_date = '2024-04-01'
end_date =  '2024-04-29'

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)

In [8]:
%%time
print(f'generating results from {start_date} to {end_date}, for {len(date_range)} days')
res =  pd.DataFrame()
for date in date_range:
    date = date.date()
    query = f"""with 
    all_cust 
    AS( 
        SELECT count(distinct cust_id) Total_registered_users
        from prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb
        where created_dt <= "{date}" and cust_id is not null ),
    open_TSA_cust
    AS(
        SELECT count (distinct OFCUSTOMERID) Total_open_TSA_customers from `core_raw.customer_accounts` 
            where CRINTERDESC like 'Transactional Savings Account Inv_R'
            and OFCUSTOMERID in (select distinct cust_id 
            from prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb where cust_id is not null)
            and OFISCLOSED = 'N'
            AND OFDATEOPENED <= "{date}"),
    loan_cust --list of customers having active loan
    AS (
        SELECT 
            distinct CAST(customerId AS STRING) customerId
        FROM 
            `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
        WHERE 
            loanPaidStatus IN ('Normal', 'In Arrears')),
    non_ACL_cust --list of customers not having ACL loans
    AS (
        SELECT 
            distinct CAST(customerId AS STRING) customerId
        FROM 
            `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
        WHERE 
            loanType != "TSBL" 
            AND date(approvedDateTime) = "{date}"),    
    MAU_with_trx 
    AS (
        SELECT DISTINCT customer_id 
        FROM `prj-prod-dataplatform.risk_mart.customer_transactions`
        WHERE transaction_date BETWEEN DATE_SUB('{date}', INTERVAL 30 DAY) AND '{date}'
        AND ABS(trx_amount) > 100
        AND account_type = "Tonik Account"
        AND customer_id NOT IN (SELECT customerId FROM loan_cust)),
    MAU_with_bal 
     AS (
        SELECT 
            DISTINCT client_id 
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_balance`
        WHERE
            clearedbalance > 100
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")
            AND balanceDateAsOf = '{date}'
            AND client_id NOT IN (SELECT customerId FROM loan_cust)),
    MAU 
        AS (
        SELECT COUNT(customer_id) AS MAU 
        FROM 
            (SELECT customer_id 
            FROM 
                MAU_with_trx
            UNION DISTINCT
            SELECT 
                client_id AS customer_id
            FROM 
                MAU_with_bal)),
    transactional_cust --Mareks criteria
    AS (
        SELECT 
            DISTINCT customer_id
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_transactions`
        WHERE
            transaction_date BETWEEN DATE_SUB('{date}', INTERVAL 3 MONTH) AND '{date}'
            AND customer_id NOT IN (SELECT customerId FROM non_ACL_cust)
        GROUP BY
            customer_id
        HAVING 
            COUNT(*) >= 5
    ),
    cust_with_balance --Mareks criteria
    AS (
        SELECT 
            DISTINCT client_id AS customer_id 
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_balance`
        WHERE
            clearedbalance > 1000
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")
            AND balanceDateAsOf = '{date}'
            AND client_id NOT IN (SELECT customerId FROM non_ACL_cust)),
    MMAU
    AS (
        SELECT 
            COUNT(customer_id) AS MMAU 
        FROM 
            ( SELECT 
            customer_id 
        FROM 
            transactional_cust
        UNION DISTINCT
        SELECT 
            customer_id
        FROM 
            cust_with_balance)),
    ACL_customers 
    AS (
        SELECT 
      count(distinct customerId ) ACL_customers
        FROM 
            `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
        WHERE 
            loanType = "TSBL" 
            AND date(approvedDateTime) = "{date}"),
    Offers_extended 
    AS (
        SELECT count( distinct cust_id) Offers_extended
    FROM 
    `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_offers_trx` 
    where date(offer_start_date) <= "{date}"),
    b
    AS (
        SELECT 
            '{date}' AS Date,
            *
        FROM 
            all_cust  
        CROSS JOIN 
            open_TSA_cust
        CROSS JOIN 
            MAU 
        CROSS JOIN 
            MMAU
        CROSS JOIN
            ACL_customers
        CROSS JOIN 
            Offers_extended)
    select Date,
            Total_registered_users,
            Total_open_TSA_customers,
            MAU,
            MMAU,
            ACL_customers,
            MMAU - ACL_customers  MMAU_ACL_difference, 
            Offers_extended,
            CASE 
                WHEN Offers_extended<> 0 then ACL_customers/Offers_extended 
                ELSE NULL END AS Uptake
            
            from b;
            """
    df = client.query(query).to_dataframe()
    res = pd.concat([res,df])


generating results from 2024-04-01 to 2024-04-29, for 29 days
CPU times: user 582 ms, sys: 138 ms, total: 720 ms
Wall time: 1min 55s


In [9]:
res

,Date,Total_registered_users,Total_open_TSA_customers,MAU,MMAU,ACL_customers,MMAU_ACL_difference,Offers_extended,Uptake
0,2024-04-01,1447123,511855,76214,58540,0,58540,0,NaN
0,2024-04-02,1448388,512541,76097,57740,0,57740,0,NaN
0,2024-04-03,1449558,513215,76012,57573,0,57573,0,NaN
0,2024-04-04,1450805,513885,75885,57497,0,57497,0,NaN
0,2024-04-05,1452013,514595,75813,57153,0,57153,0,NaN
0,2024-04-06,1453178,515363,75700,57248,0,57248,0,NaN
0,2024-04-07,1454422,516236,75631,57464,0,57464,0,NaN
0,2024-04-08,1455814,516988,75232,57943,0,57943,0,NaN
0,2024-04-09,1457146,517742,75186,58455,0,58455,0,NaN
0,2024-04-10,1458491,518476,75140,57383,0,57383,0,NaN


In [10]:
res.to_excel(f"MMAU_from_{start_date.replace('-','_')}.xlsx",index=False)